# Mushroom Classification

* 식용/독 버섯 분류 문제를 하던 도중 feature select 에 대한 좋은 코드를 볼 수 있어 학습을 시작하게 되었습니다. <br> feature 를 선택하는 기술? 방법에는 여러가지가 있습니다.

1. Filter methods
2. Wrapper methods
3. Embedded methods

* Filter methods

1. Basic methods (코드 구현)
2. Univariate methods
3. Information gain 
4. Fischer score
5. Correlation Matrix with Heatmap (코드 구현)

* Wrapper methods 

1. Forward Selection
2. Backward Elimination
3. Exhaustive Feature Selection
4. Recursive Feature Elimination
5. Recursive Feature Elimination with Cross-Validation (코드 구현)

* Embedded methods

1. LASSO
2. RIDGE
3. Tree Importance

* Randomforest importance (코드 구현)

- --

* 실제 Mushroom classification https://www.kaggle.com/uciml/mushroom-classification 문제에서 모든 feature 를 넣고 RandomForest model 을 돌리면 <br> 100%(Cross Voildation - size 0.2)로 독버섯과 식용버섯을 분류해준다. <br> 랜덤하게 feature 를 절반정도 drop 해도 100% 에 근접하게 식용/독버섯을 분류함

* 해당 데이터 셋에서 우리에게 말하고 싶은 내용 중 하나는 다음과 같다.

Which features are most indicative of a poisonous mushroom?
* 그럴만도하다. 대충 데이터 셋을 꾸겨넣어도 성능이 굉장히 뛰어나다. 그렇다면 우리는 효율을 따져야하지 않을까? 적은 자원을 투입해서 성능 내기

In [2]:
import numpy as np
import pandas as pd

- --
## Filter Methods

* Remove constant features 

In [1]:
from sklearn.feature_selection import VarianceThreshold

In [69]:
X = [[0, 2, 0, 3], [0, 1, 4, 3], [0, 1, 1, 3]]
selector = VarianceThreshold()
selector.fit_transform(X)


array([[2, 0],
       [1, 4],
       [1, 1]])

In [70]:
X = [[0, 2, 0, 3], [0, 1, 4, 3], [0, 1, 1, 3]]
selector = VarianceThreshold(threshold=0)
selector.fit(X)
selector.get_support() # 중요도가 높은 것만 True로 반영해줌 VarianceThreshold method 의 파리미터 threshold 의 default = 0 

array([False,  True,  True, False])

In [71]:
sum(selector.get_support())

2

In [72]:
dataset = pd.read_csv('mushrooms.csv') # 실전

dataset = dataset.drop('class', axis=1) # 타겟은 제외
dataset.shape

(8124, 22)

In [73]:
dataset = pd.get_dummies(dataset) # VarianceThreshold 매서드를 사용하려면 value 가 float 형이어야함
dataset.shape

(8124, 117)

In [74]:
selector = VarianceThreshold()
selector.fit_transform(dataset)

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [75]:
selector = VarianceThreshold()
selector.fit(dataset)
selector.get_support() # get_support 는 선택된 feature 를 bool type 으로 반환해준다. 즉 false = Drop / True = Select

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [66]:
sum(selector.get_support()) # 이 말은 즉 117 개중 116 개는 사용해야 한다는 것이다. 제외되는 1 가지가 있는데 분명 해당 Feature 의 value 는 한 가지일 것이다. 그래서 제외


116

In [80]:
for col in dataset.columns: # 어떤 feature 가 제외됐는지 확인
    if col not in dataset.columns[selector.get_support()]:
        print(col) # 제외된 feature 은 veil-type / veil-type_p 라고 출력되는 이유는 더미화로 그렇다. 아래서 다시 한 번 보자.

veil-type_p


In [67]:
# 제외되는 feauture 은 veil type

dataset = pd.read_csv('mushrooms.csv')

dataset['veil-type'].value_counts() # value 가 p 로 한가지다. 모든 sample 의 veil-type 이 P 인데 굳이 넣을 필요가 없다는 의미다.

p    8124
Name: veil-type, dtype: int64

* Remove quasi-constant features

In [81]:
dataset = pd.read_csv('mushrooms.csv') # 실전

dataset = dataset.drop('class', axis=1) # 타겟은 제외
dataset.shape

(8124, 22)

In [82]:
dataset = pd.get_dummies(dataset) # VarianceThreshold 매서드를 사용하려면 value 가 float 형이어야함
dataset.shape

(8124, 117)

In [84]:
selector = VarianceThreshold(threshold=0.01) # 임계점을 넘지 못하는 feature 은 모두 제외
sum(selector.get_support()) # 24 개의 features 가 제외됨 

93

In [ ]:
# 제외된 이유는 아래서 설명

In [90]:
for col in dataset.columns: # 제외된 features
    if col not in dataset.columns[selector.get_support()]:
        print(col) 

cap-shape_c
cap-shape_s
cap-surface_g
cap-color_c
cap-color_r
cap-color_u
odor_m
gill-color_o
gill-color_r
stalk-surface-above-ring_y
stalk-color-above-ring_c
stalk-color-above-ring_y
stalk-color-below-ring_c
stalk-color-below-ring_y
veil-type_p
veil-color_y
ring-number_n
ring-type_f
ring-type_n
spore-print-color_b
spore-print-color_o
spore-print-color_r
spore-print-color_u
spore-print-color_y


In [97]:
dataset['cap-shape_c'].value_counts() / len(dataset) # 제외된 feautures 구성비

0    0.999508
1    0.000492
Name: cap-shape_c, dtype: float64

In [100]:
dataset['habitat_u'].value_counts() / len(dataset) # 제외되지 않은 features 구성비

0    0.954702
1    0.045298
Name: habitat_u, dtype: float64